In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

def escraper(initial, supreme, waiting_time):
    animes = {"ID": [], "Title": [], "Synopsis": [], "Score":[]}
    for i in range(initial, supreme):
        url = "https://myanimelist.net/anime/"+str(i)
        time.sleep(waiting_time)
        page = requests.get(url)
        soup = BeautifulSoup(page.text,"html.parser")
        certificate = soup.find(text="This page doesn't exist.")
        if certificate == None:
            print(f"Anime {i} successfully retrieved!")
            meta_description_syn = soup.find("meta", property="og:description")
            if meta_description_syn == None: #Then we overloaded MAL.
                print(f"But synopsis is unreachable for some reason? (Restart kernel)")
                anim = pd.DataFrame.from_dict(animes)
                anim.to_csv(str(initial)+'Mallanimelist.csv')
                break
            else:
                animes["ID"].append(i)
                sainopsis = meta_description_syn.get("content")
                animes["Synopsis"].append(sainopsis)
        
                tairul = soup.find_all("title")
                titulo = re.sub("<title>", '', str(tairul[0]))
                titulo = re.sub(" - MyAnimeList.net", '', titulo)
                titulo = re.sub("</title>", '', titulo)
                titulo = re.sub("\n", '', titulo)
                animes["Title"].append(titulo)

                score_element = soup.find("span", class_="score-label")
                scor = score_element.text.strip()
                animes["Score"].append(scor)
    anim = pd.DataFrame.from_dict(animes)
    anim.to_csv(str(initial)+'Mallanimelist.csv')

HOW TO GET THE supreme VALUE:

1.- Go to: https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1

2.- Obtain the last anime till today. September 6, 2023 is Longzhu_Xun_Youji with ID 56559

3.- Round up that number to taste. In my particular case, I picked supreme = 56560 since I made my run just few hours after my query (at most, one new anime might be uploaded)

4.- WARNING: Don't pick a brutally upper bound since the program will keep running until reaching that upper bound.
             Don't waste resources: every python run kills at least one Koala, so make their lives worth it (poor Koalas).

In [ ]:
waiting_time = 3
initial = 28892 #693 #127 #28891 (internet gonne for a while)
supreme = 56560

In [ ]:
import math

#Ya escrapeamos todos los animes antes de 28891.
tamaño_segmento = 500
mumero = (56600 - 28891)
a_segmentar = math.ceil(mumero/tamaño_segmento) #Cuántos segmentos hay de tamaño 500, redondeado hacia arriba porsi las porsis.
print(a_segmentar)

for i in range(a_segmentar): #Checamos los 56 segmentos de tamaño 500 que hay entre 56600 y 28891
    inicio = 28891+i*(tamaño_segmento)+1
    final = 28891+(i+1)*(tamaño_segmento)
    print("********************")
    print(f"Starts in: {inicio}")
    print(f"Ends in: {final}")
    print("********************")
    escraper(inicio, final, waiting_time)

#####ENGLISH NOTES:
#Segmento is used in the same sense as Batch. so tamaño_segmento == batch_size.